In [1]:
import os
print(os.getcwd())
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)
print(os.getcwd())

C:\Users\danie\Documents\COGS402\cogs402longformer\Huggingfacetutorial
C:\Users\danie\Documents\COGS402\cogs402longformer


In [2]:
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig

from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer, LongformerConfig
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = 'models/longformer-finetuned_papers/checkpoint-2356'

# load model
model = LongformerForSequenceClassification.from_pretrained(model_path, num_labels = 2)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

In [5]:
def predict(inputs, token_type_ids=None, position_ids=None):
    output = model(inputs, token_type_ids=token_type_ids,
                 position_ids=position_ids)
    return output.logits

In [6]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [7]:
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, truncation = True, add_special_tokens=False, max_length = 256)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

In [8]:
def custom_forward(inputs):
    preds = predict(inputs)
    print(torch.softmax(preds, dim = 1)[0][0])
    return torch.softmax(preds, dim = 1)[0][0].unsqueeze(0)

In [9]:
lig = LayerIntegratedGradients(custom_forward, model.longformer.embeddings)

In [10]:
from datasets import load_dataset
cogs402_ds = load_dataset("danielhou13/cogs402dataset")["test"]

Using custom data configuration danielhou13--cogs402dataset-5c7aa10e6c95142f
Reusing dataset parquet (C:\Users\danie\.cache\huggingface\datasets\parquet\danielhou13--cogs402dataset-5c7aa10e6c95142f\0.0.0\0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
testval = 125
text = cogs402_ds['text'][testval]
label = cogs402_ds['labels'][testval]
print(label)

1


In [12]:
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [13]:
# model(input_ids)

In [14]:
# predict(input_ids)

In [15]:
# custom_forward(input_ids)

In [16]:
attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    return_convergence_delta=True,
                                    internal_batch_size = 2)

tensor(0.0005, device='cuda:0')
tensor(0.9026, device='cuda:0')
tensor(0.9030, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.9057, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.9108, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.9172, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.9145, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.9062, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.9069, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.9111, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.9213, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.9489, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.9607, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.9601, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.9562, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.9524, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.9587, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(0.9813, device='cuda:0', grad_fn=<SelectBack

In [17]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [18]:
attributions_sum = summarize_attributions(attributions)

In [19]:
score = predict(input_ids)

In [20]:
# storing couple samples in an array for visualization purposes
score_vis = viz.VisualizationDataRecord(
                        attributions_sum,
                        torch.softmax(score, dim = 1)[0][0],
                        torch.argmax(torch.softmax(score, dim = 1)[0]),
                        label,
                        text,
                        attributions_sum.sum(),       
                        all_tokens,
                        delta)

print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_vis])

 Visualization For Score 
